In [1]:
from numpy.random import seed
import os
import random
import numpy as np
import skimage
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from keras import backend as keras
from utils import DataGenerator
from unet import *
from main import showHistory, TrainValTensorBoard


2023-05-12 17:31:56.597677: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 17:31:56.792036: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-12 17:31:57.604927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-12 17:31:57.605004: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
params = {'batch_size':1,
          'dim':(128,128,128),
          'n_channels':1,
          'shuffle': True}
seismPathT = "../data_seismic/train/seis/"
faultPathT = "../data_seismic/train/fault/"
seismPathV = "../data_seismic/validation/seis/"
faultPathV = "../data_seismic/validation/fault/"
train_ID = range(200)
valid_ID = range(20)
train_generator = DataGenerator(dpath=seismPathT,fpath=faultPathT,
                                data_IDs=train_ID,**params)
valid_generator = DataGenerator(dpath=seismPathV,fpath=faultPathV,
                                data_IDs=valid_ID,**params)
model = unet(input_size=(None, None, None,1))
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

2023-05-12 17:32:00.132102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5680 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:73:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d (Conv3D)                (None, None, None,   448         ['input_1[0][0]']                
                                None, 16)                                                         
                                                                                                  
 conv3d_1 (Conv3D)              (None, None, None,   6928        ['conv3d[0][0]']                 
                                None, 16)                                                     

/home/dnlab/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [3]:
# checkpoint
filepath="check1/fseg-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
      verbose=1, save_best_only=False, mode='max')
logging = TrainValTensorBoard()
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, 
#                              patience=20, min_lr=1e-8)
callbacks_list = [checkpoint, logging]
print("data prepared, ready to train!")
# Fit the model
history=model.fit_generator(generator=train_generator,
validation_data=valid_generator,epochs=100,callbacks=callbacks_list,verbose=1)
model.save('check1/fseg.hdf5')

/tmp/ipykernel_63534/2287510817.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator=train_generator,


data prepared, ready to train!
Epoch 1/100


2023-05-12 12:56:38.892664: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101


200/200 [==============================] - ETA: 0s - loss: 0.2779 - accuracy: 0.9283
Epoch 1: saving model to check1/fseg-01.hdf5
check point
200/200 [==============================] - 170s 797ms/step - loss: 0.2779 - accuracy: 0.9283 - val_loss: 0.2690 - val_accuracy: 0.9277
Epoch 2/100
200/200 [==============================] - ETA: 0s - loss: 0.2408 - accuracy: 0.9299
Epoch 2: saving model to check1/fseg-02.hdf5
check point
200/200 [==============================] - 159s 796ms/step - loss: 0.2408 - accuracy: 0.9299 - val_loss: 0.2348 - val_accuracy: 0.9277
Epoch 3/100
200/200 [==============================] - ETA: 0s - loss: 0.2091 - accuracy: 0.9307
Epoch 3: saving model to check1/fseg-03.hdf5
check point
200/200 [==============================] - 160s 797ms/step - loss: 0.2091 - accuracy: 0.9307 - val_loss: 0.1764 - val_accuracy: 0.9336
Epoch 4/100
200/200 [==============================] - ETA: 0s - loss: 0.1756 - accuracy: 0.9367
Epoch 4: saving model to check1/fseg-04.hdf5
che

KeyError: 'acc'

<Figure size 1000x600 with 0 Axes>

In [3]:
showHistory(history)

NameError: name 'history' is not defined